In [45]:
import jieba
import tensorflow as tf
import random
import numpy as np
from collections import Counter
import time

In [46]:
with open('Javasplittedwords.txt', encoding='utf8') as f:
    text = f.read()
words = text.split(' ')

In [47]:
words_count = Counter(words)
# 过滤出现次数太少的词语
words = [w for w in words if words_count[w] > 50]

In [48]:
vocab = set(words) # 去重
vocab_to_int = {w:c for c, w in enumerate(vocab)}
int_to_vocab = {c:w for c, w in enumerate(vocab)}

In [49]:
print("total words: {}".format(len(words)))
print("unique words: {}".format(len(vocab)))

total words: 8614953
unique words: 6846


## 对原文本进行vocab到int转换

In [50]:
int_words = [vocab_to_int[w] for w in words]

## 采样

过滤一些出现次数太高的词语， 过滤公式是

In [51]:
t = 1e-5
threshold = 0.9

int_word_counts = Counter(int_words)
# 词语总量：8614953
total_count = len(int_words)
word_freqs = {w: c/total_count for w, c in int_word_counts.items()}
prob_drop = {w:1 - np.sqrt(t / word_freqs[w]) for w in int_word_counts}
train_words = [w for w in int_words if prob_drop[w] < threshold]

In [52]:
len(train_words)

3905916

In [53]:
def get_target(words, idx, window_size=5):
    """
    获取input word的上下文单词列表
    参数：
        words: 单词列表
        idx: index word的索引号
        window_size: 窗口大小
    """
    target_window = np.random.randint(1, window_size + 1)
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    targets = set(words[start_point:idx] + words[idx + 1: end_point + 1])
    return list(targets)

In [54]:
def get_batches(words, batch_size, window_size = 5):
    """
    构造一个batch生成器
    """
    n_batchs = len(words) // batch_size
    # 零头就不算了
    words = words[:n_batchs * batch_size]
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx : idx + batch_size]
        for i in range(len(batch)):
            batch_x = batch[i]
            batch_y = get_target(batch, i, window_size)
            # 一个input word会对应多个output word，因此需要长度统一
            x.extend([batch_x] * len(batch_y))
            y.extend(batch_y)
        yield x, y
        

## 构建网络

In [55]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')

In [56]:
vocab_size = len(int_to_vocab)
embedding_size = 200
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    # 实现lookup
    embed = tf.nn.embedding_lookup(embedding, inputs)

# 负采样

In [57]:
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(vocab_size))
    
    # 定义损失函数
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, vocab_size)
    # 在对应维度求平均值
    cost = tf.reduce_mean(loss)
    # 优化器
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    

# 验证

In [58]:
with train_graph.as_default():
    # 选七个词来验证
    valid_size = 7
    valid_window = 100
    # 0-100 中随机选三个
    valid_examples = np.array(random.sample(range(valid_window), valid_size // 2))
    # 1000， 1100中随机选三个
    valid_examples = np.append(valid_examples, random.sample(range(1000, 1000+valid_window), valid_size // 2))
    valid_examples = [
        vocab_to_int['word'],
        vocab_to_int['ppt'],
        vocab_to_int['熟悉'],
        vocab_to_int['java'],
        vocab_to_int['能力'],
        vocab_to_int['逻辑思维'],
        vocab_to_int['了解']
    ]
    
    valid_size = len(valid_examples)
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    ## 计算每个单词的模，并进行单位化
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
    normalized_embedding = embedding / norm
    # 查找验证单词的词向量
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    # 计算余弦相似度
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [60]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())
    
    for e in range(1, epochs + 1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            feed = {inputs:x,
                    labels:np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            loss += train_loss
            
            if iteration % 100 == 0:
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log = "Nearest to [{}]:".format(valid_word)
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = "{} {},".format(log, close_word)
                print(log)
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

KeyboardInterrupt: 